In [10]:
import numpy as np
## Progress bar
from tqdm.auto import tqdm
import pylab as plt
import copy
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks, regularizers
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
if 'google.colab' in str(get_ipython()):
  print("All ok -- code is running on a Google Colab")
else:
    print("==="*15, " WARNING ","==="*15)
    print("For DSA4212 assignment 1, code needs to be run on a Google Colab with a single GPU")
    print("==="*15, " WARNING ","==="*15)



All ok -- code is running on a Google Colab


In [12]:
# mount the Google Drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# goto to data folder -- you may need to change this location
%cd /content/drive/MyDrive/DSA4212/assignment_1_script/

/content/drive/MyDrive/DSA4212/assignment_1_script


In [14]:
class_names=["fish", "dog", "device", "chainsaw", "church", "horn", "truck", "petrol", "golf", "parachute"]

# load data -- be patient, does take a few secs
data_train_all = np.load("assignment_1_train.npz")
data_test = np.load("assignment_1_test.npz")

X_train_all = data_train_all["img"].astype(np.float32) / 255.   # set pixel intensities to [0,1]
X_test = data_test["img"].astype(np.float32) / 255.
Y_train_all = data_train_all["label"].astype(int)
Y_test = data_test["label"].astype(int)

_,H,W,C = X_train_all.shape
print(f"Img Shape: {H}x{W}x{C}")

Img Shape: 128x128x3


In [6]:
# shuffle the train set
indices_shuffled = np.arange(len(X_train_all))
np.random.shuffle(indices_shuffled)
X_train_all = X_train_all[indices_shuffled]
Y_train_all = Y_train_all[indices_shuffled]

# shuffle the validation set
indices_shuffled = np.arange(len(X_test))
np.random.shuffle(indices_shuffled)
X_test = X_test[indices_shuffled]
Y_test = Y_test[indices_shuffled]

In [7]:
len_train_all = len(X_train_all)
len_train = int(0.8 * len_train_all)
len_val = len_train_all - len_train
len_tets = len(X_test)

X_train = X_train_all[:len_train]
Y_train = Y_train_all[:len_train]

X_val = X_train_all[len_train:]
Y_val = Y_train_all[len_train:]

In [ ]:
X_train.shape

(7436, 128, 128, 3)

In [ ]:
# Un-augmented training images with batch size = 32
default_gen = keras.preprocessing.image.ImageDataGenerator().flow(
    X_train, Y_train,
    batch_size=32)

# Un-augmented validation images with batch size = 32
val_generator = keras.preprocessing.image.ImageDataGenerator().flow(
    X_val, Y_val,
    batch_size=32)

# Image augmentation; feeds the images into this generator which augments it
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

# Augmented training images with batch size = 32
train_generator = train_datagen.flow(
    X_train, Y_train,
    batch_size=32)

In [ ]:
# Basic CNN model
BCCN = models.Sequential()
BCCN.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
BCCN.add(layers.MaxPooling2D((2, 2)))
BCCN.add(layers.Conv2D(64, (3, 3), activation='relu'))
BCCN.add(layers.MaxPooling2D((2, 2)))
BCCN.add(layers.Conv2D(128, (3, 3), activation='relu'))
BCCN.add(layers.Flatten())
BCCN.add(layers.Dense(64, activation='relu'))
BCCN.add(layers.Dropout(0.2))
BCCN.add(layers.Dense(10,activation = "softmax"))

BCCN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

BCCN.fit(default_gen, validation_data = val_generator, epochs=10)

test_loss, test_acc = BCCN.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', test_acc*100, "%")

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


233/233 [==============================] - 6s 18ms/step - loss: 2.0680 - accuracy: 0.2600
Epoch 2/10
233/233 [==============================] - 4s 18ms/step - loss: 1.6182 - accuracy: 0.4566
Epoch 3/10
233/233 [==============================] - 5s 20ms/step - loss: 1.3049 - accuracy: 0.5666
Epoch 4/10
233/233 [==============================] - 4s 18ms/step - loss: 1.0360 - accuracy: 0.6607
Epoch 5/10
233/233 [==============================] - 4s 18ms/step - loss: 0.7294 - accuracy: 0.7562
Epoch 6/10
233/233 [==============================] - 5s 20ms/step - loss: 0.5180 - accuracy: 0.8253
Epoch 7/10
233/233 [==============================] - 4s 18ms/step - loss: 0.3711 - accuracy: 0.8778
Epoch 8/10
233/233 [==============================] - 4s 18ms/step - loss: 0.2761 - accuracy: 0.9072
Epoch 9/10
233/233 [==============================] - 4s 19ms/step - loss: 0.2133 - accuracy: 0.9264
Epoch 10/10
233/233 [==============================] - 4s 19ms/step - loss: 0.1781 - accuracy: 0.9411


In [ ]:
# Base LeNet model
BLN = models.Sequential()
BLN.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
BLN.add(layers.AveragePooling2D((2,2)))
BLN.add(layers.Activation('sigmoid'))
BLN.add(layers.Conv2D(64,(3,3), activation='relu'))
BLN.add(layers.AveragePooling2D((2,2)))
BLN.add(layers.Activation('sigmoid'))
BLN.add(layers.Conv2D(128, (3,3), activation='relu'))
BLN.add(layers.Flatten())
BLN.add(layers.Dense(84, activation='relu'))
BLN.add(layers.Dense(10))


BLN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

BLN.fit(default_gen, validation_data = val_generator, epochs=10)

test_loss, test_acc = BLN.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', test_acc*100, "%")

In [ ]:
# Base AlexNet
AlexNet = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), activation='relu', input_shape=(128,128,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

AlexNet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

AlexNet.fit(default_gen, validation_data = val_generator, epochs=10)

test_loss, test_acc = AlexNet.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', test_acc*100, "%")

In [ ]:
## VGG 16

features = [64,128]
VGG=keras.models.Sequential()

VGG.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
#pooling layer
VGG.add(layers.MaxPooling2D(2,2))
VGG.add(layers.BatchNormalization()) 
for i in features:

#covolution layer
  VGG.add(layers.Conv2D(i,(3,3),activation='relu'))
#pooling layer
  VGG.add(layers.MaxPooling2D(2,2))
  VGG.add(layers.BatchNormalization())

VGG.add(layers.Flatten())
#o/p layer
VGG.add(layers.Dense(10,activation='softmax'))

#covolution layer
#VGG.add(layers.Conv2D(features[1],(3,3),activation='relu'))
#pooling layer
#VGG.add(layers.MaxPooling2D(2,2))
#VGG.add(layers.BatchNormalization())
#covolution layer
#VGG.add(layers.Conv2D(features[2],(3,3),activation='relu'))
#pooling layer
#VGG.add(layers.MaxPooling2D(2,2))
#VGG.add(layers.BatchNormalization())
#covolution layer
#VGG.add(layers.Conv2D(features[3],(3,3),activation='relu'))
#pooling layer
#VGG.add(layers.MaxPooling2D(2,2))
#VGG.add(layers.BatchNormalization())
#i/p layer
#VGG.add(layers.Flatten())
#o/p layer
#VGG.add(layers.Dense(10,activation='softmax'))

opt = keras.optimizers.Adamax(learning_rate=0.0001)

VGG.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#VGG.summary()
VGG.fit(default_gen, validation_data = val_generator, epochs=10)

val_loss, val_acc = VGG.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', val_acc*100, "%")

## Initialisation stage ##

- Will be using the VGG architecture as a reference
- Constant learning rate of 0.001
- Adam as optimiser
- 3 layers for now with features, [32,48,64]
- Activation function will be sigmoid


These initial model settings are from the settings that yielded the highest validation accuracy.

In [8]:
## VGG 16

features = [48,64]
VGG=keras.models.Sequential()

VGG.add(layers.Conv2D(32,(3,3),activation='sigmoid',input_shape=(128,128,3)))
#pooling layer
VGG.add(layers.MaxPooling2D(2,2))
VGG.add(layers.BatchNormalization()) 
for i in features:

#covolution layer
  VGG.add(layers.Conv2D(i,(3,3),activation='sigmoid'))
#pooling layer
  VGG.add(layers.MaxPooling2D(2,2))
  VGG.add(layers.BatchNormalization())

VGG.add(layers.Flatten())
#o/p layer
VGG.add(layers.Dense(10,activation='softmax'))


opt = keras.optimizers.Adam(learning_rate=0.001)

VGG.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#VGG.summary()
VGG.fit(default_gen, validation_data = val_generator, epochs=10)

val_loss, val_acc = VGG.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', val_acc*100, "%")

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 39/233 [====>.........................] - ETA: 3:28 - loss: 3.2792 - accuracy: 0.2452

KeyboardInterrupt: ignored

## Overfit ##
- Play around with the parameters to maximise training accuracy and at the same time increase validation accuracy.
- Overfitted model specifications
1. Activation function: Relu
2. Kernel size : (3,3)
3. Features : [32,48,64,128]
4. Optimiser: Adam
5. Lr_schedule : Exponential 
6. Initial LR : 0.0001

- Training accuracy by 10th epoch : 0.9833
- Validation accuracy : 0.6339
- Training time: 1m 32s


In [ ]:
## VGG 16
# Changed optimiser to adam :57%
# Changed max_pooling window (3,3):54%
# Changed activation function to relu: 59%
# Change features to [32,48,64,128]: 63%
# Change to a changing learning rate: 65.91%
# Change kernel size to (4,4): 63.9%
# Change kernel size to (2,2): 65.69%
# Adding one more dense layer: 64.999%


features = [48,64,128,156]
VGG=keras.models.Sequential()

VGG.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
#pooling layer
VGG.add(layers.MaxPooling2D(2,2))
VGG.add(layers.BatchNormalization()) 
for i in features:

#covolution layer
  VGG.add(layers.Conv2D(i,(3,3),activation='relu'))
#pooling layer
  VGG.add(layers.MaxPooling2D(2,2))
  VGG.add(layers.BatchNormalization())

VGG.add(layers.Flatten())
#o/p layer
VGG.add(layers.Dense(10,activation='softmax'))

initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

opt = keras.optimizers.Adam(learning_rate=lr_schedule)

VGG.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#VGG.summary()
VGG.fit(default_gen, validation_data = val_generator, epochs=10)

val_loss, val_acc = VGG.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', val_acc*100, "%")

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


233/233 [==============================] - 17s 20ms/step - loss: 1.6029 - accuracy: 0.4852
Epoch 2/10
233/233 [==============================] - 5s 22ms/step - loss: 1.0711 - accuracy: 0.6555
Epoch 3/10
233/233 [==============================] - 5s 20ms/step - loss: 0.8401 - accuracy: 0.7281
Epoch 4/10
233/233 [==============================] - 5s 20ms/step - loss: 0.6581 - accuracy: 0.7883
Epoch 5/10
233/233 [==============================] - 5s 23ms/step - loss: 0.4907 - accuracy: 0.8433
Epoch 6/10
233/233 [==============================] - 5s 20ms/step - loss: 0.3293 - accuracy: 0.9024
Epoch 7/10
233/233 [==============================] - 5s 20ms/step - loss: 0.2050 - accuracy: 0.9465
Epoch 8/10
233/233 [==============================] - 5s 23ms/step - loss: 0.1408 - accuracy: 0.9630
Epoch 9/10
233/233 [==============================] - 5s 20ms/step - loss: 0.0931 - accuracy: 0.9770
Epoch 10/10
233/233 [==============================] - 5s 20ms/step - loss: 0.0686 - accuracy: 0.9833

## Regularization ##
- Implementing restrictions and reducing capacity to increase validation accuracy at the expense of training accuracy
- Dereasing batch size
- Add dropout layer to each iteration
- Add dropout layer in input stage
- Early stopping
- L2 and L1 regularisation

Final model specifications
1. Dropout layer at input stage (0.25)
2. L2 regularisation at final dense layer (0.01)
3. Early stopping (patience = 5, min_delta = 0.1)
4. No batch size specification

- Training accuracy by 10th epoch : 0.9627 (A lot of room to take increase validation accuracy)
- Training time : 1m28s
- Validation accuracy: 68.226%

In [9]:
## VGG 16
## Using dropout layer per iteration
# Add dropout layer(0.5) at each iteration: 60.5%
# Add dropout layer(0.1) at each iteration: 67.688%
# Add dropout layer(0.2) at each iteration: 69.95%
# Add dropout layer(0.3) at each iteration: 68.33%
# Add dropout layer(0.25) at each iteration: 71.77% , training accuracy: 0.757

## Using dropout layer for first layer(0.25):68.27% , training accuracy: 0.9662

## Decreasing batch size(Doesnt seem to do any good)
# Batch_size 256 : 15%
# Batch_size 750 : 13%  


## Early stopping 
# Patience(5), min_delta(0.001) : 64.46% 56s
# Patience(5), min_delta(0.01): 65.6989% 1m30s
# Patience(5), min_delta(0.1): 65.86% 57s

## L2 regularisation --> Will use this for grid search later
# (0.001) 67.58%

## L1 regularisation --> Will use this for grid search later
# (0.001) 67.15%


features = [48,64,128,156]
VGG=keras.models.Sequential()

VGG.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
#pooling layer
VGG.add(layers.MaxPooling2D(2,2))
VGG.add(layers.BatchNormalization()) 
VGG.add(layers.Dropout(0.25))
for i in features:

#covolution layer
  VGG.add(layers.Conv2D(i,(3,3),activation='relu'))
#pooling layer
  VGG.add(layers.MaxPooling2D(2,2))
  VGG.add(layers.BatchNormalization())
#  VGG.add(layers.Dropout(0.25))

VGG.add(layers.Flatten())
#o/p layer
VGG.add(layers.Dense(10,activation='softmax',kernel_regularizer=regularizers.l2(0.001)))

early_stopping = callbacks.EarlyStopping(
    min_delta=0.1, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
)

initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

opt = keras.optimizers.Adam(learning_rate=lr_schedule)

VGG.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#VGG.summary()
VGG.fit(default_gen, validation_data = val_generator, epochs=10,callbacks=[early_stopping])

val_loss, val_acc = VGG.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', val_acc*100, "%")

Epoch 1/10
145/233 [=================>............] - ETA: 1:40 - loss: 1.8178 - accuracy: 0.4256

KeyboardInterrupt: ignored

## Tuning parameters ##
- The four hyperparameters that will require tuning will be min_delta, lambda and dropout rate.
- This stage is about finding the best parameters that returns the best validation accuracy.

1. Normal grid search

In [ ]:
param_grid = dict(rate=[0.1,0.2,0.3,0.4,0.5],min_delta = [0.1,0.01,0.001,0.0001],l2 = [0.1,0.1,0.001,0.0001])
grid = GridSearchCV(estimator=VGG, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

NameError: ignored